Imports:

In [2]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import random as r
import sklearn
from sklearn.utils.random import sample_without_replacement

random_state = 12

Convert 3d data to png images:

In [2]:
# Path to the directory containing your images and labels
images_dir = 'data/imagesTr'
labels_dir = 'data/labelsTr'
output_dir_images = 'data/images_png' 
output_dir_labels = 'data/labels_png'
positive_images = []
negative_images = []
positive_labels = []
negative_labels = []

os.makedirs(output_dir_labels, exist_ok=True)
os.makedirs(output_dir_images, exist_ok=True)

# List all the .nii.gz files in the directory
file_list = [file for file in os.listdir(images_dir) if file.endswith('.nii.gz') and file.startswith('lung')]
modified_list = [file_name[:-7] for file_name in file_list]
len_list = len(modified_list)

for idx, file in enumerate(file_list):
    print(f'\r{idx}/{len(file_list)}', end='')
    # Load the label
    label_path = os.path.join(labels_dir, file)
    label = nib.load(label_path).get_fdata()

    # Check if the label contains any white (tumor) pixels
    if np.max(label) > 0:
        # Load the corresponding image
        image_path = os.path.join(images_dir, file)
        img = nib.load(image_path).get_fdata()

        # Convert slices with tumor labels to PNG
        for i in range(label.shape[2]):  # Loop through the slices
            label_slice = label[:, :, i]
            ones = np.sum(label_slice == 1.0)

            slice_img = ((img[:, :, i] - np.min(img[:, :, i])) / (np.max(img[:, :, i]) - np.min(img[:, :, i]))) * 255.0
            slice_img = slice_img.astype(np.uint8)
            if ones > 0.015 * label_slice.shape[0]:
                positive_images.append(slice_img)
                positive_labels.append(label_slice)
            else:
                negative_images.append(slice_img)
                negative_labels.append(label_slice)

NameError: name 'os' is not defined

In [ ]:
sampled_positive_labels, sampled_positive_images = sklearn.utils.resample((negative_labels, negative_images), n_samples=len(positive_labels), random_state=random_state)
data_images = np.concatenate(positive_images, negative_images)
data_labels = np.concatenate(negative_labels, positive_labels)
sklearn.utils.shuffle((data_labels, data_images), random_state=random_state)
for idx, img in enumerate(data_images):
    output_fn = os.path.join(output_dir_images, ("image" + str(idx) + ".png"))
    plt.imsave(output_fn, img)

for idx, label in enumerate(data_labels):
    output_fn = os.path.join(output_dir_labels, ("label" + str(idx) + ".png"))
    plt.imsave(output_fn, label)